In [1]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv("./.env", override=True)

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


## Langchain에서 데이터를 효과적으로 전달하는 방법
1. RunnablePassthrough
2. RunnableParallel
3. RunnableLambda


### 1. `RunnablePassthrough`: 데이터를 그대로 넘겨(통과시켜)주는 역할

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{word}를 영어로?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm
chain

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={}, template='{word}를 영어로?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001FCEB3598E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FC85D2F590>, root_client=<openai.OpenAI object at 0x000001FCEB358830>, root_async_client=<openai.AsyncOpenAI object at 0x000001FCEB3591F0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 ***딕셔너리***

In [6]:
# chain 을 실행합니다.
chain.invoke({"word": '사과'})

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJsa21wGZwlaRI9UmSizK0JziHhNX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd87b7ea-660c-41ef-872b-224841af288e-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

But, 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능

In [7]:
# chain 을 실행합니다.
chain.invoke('사과')

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJsa5DIaZOhM11vVupxYLf9JroWn5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8692d208-eacf-42dc-b929-f846fe28c6a6-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 는 runnable 객체이며, runnable 객체는 invoke() 메소드를 사용하여 별도 실행이 가능
# RunnablePassthrough()를 사용하여 체인 구성
runnable_chain = {"word": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경됨
runnable_chain.invoke('인공지능')

AIMessage(content='Artificial Intelligence (AI)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 17, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJsbe4arT6MEI6LJDodsGM2jbj9gc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e3a399ff-1f1c-4c34-98c4-b89054310ba3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 6, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### [실습] RunnablePassthrough를 사용하여 값을 넘겨주는 체인 완성하기
- 주제: 특정 주제에 대한 인스타그램 홍보 문구 만드는 챗봇 만들기
  
1. `topic` 변수를 담은 프롬프트 템플릿 객체 생성하기
2. `marketing_chain` 을 만들어 `topic` 변수에 RunnablePassthrough()를 사용한 채인 만들기
3. `invoke()` 함수를 사용하여 `AI 챗봇` 라는 값을 넣어 체인 실행해보기

In [9]:
# 실습 코드 
prompt = PromptTemplate.from_template("{topic} 주제에 대한 인스타그램 홍보 문구 작성")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
marketing_chain = {"topic": RunnablePassthrough()} | prompt | llm
marketing_chain.invoke("AI 챗봇")

AIMessage(content='"인공지능 챗봇과 대화하며 새로운 세계를 경험해보세요! 🤖💬 #AI챗봇 #인공지능 #대화파트너 #편리한상담"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 33, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJsfp5j2xQWT9kg7Yhe2fsjlyX4kG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d2400fe3-e05a-41cf-bb98-9eadf2de56df-0', usage_metadata={'input_tokens': 33, 'output_tokens': 71, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 2. `RunnableParallel`: 여러 작업을 동시에(병렬)로 처리하도록 도와주는 도구

In [10]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [11]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울특별시입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 19, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJslUjPIztWsf5e0cy2o8J65XU8jB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b23ca6d1-0219-46b1-9f14-12733628dbe1-0', usage_metadata={'input_tokens': 19, 'output_tokens': 10, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363 km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tokens': 

#### [실습] RunnableParallel을 사용하여 두 체인을 병렬로 실행해보기
- 주제: 특정 문장에 대한 충청도 사투리/ 경상도 사투리 챗봇 만들기
- RunnablePassthrough()와 RunnableParallel() 동시에 사용하기
- model은 `gpt-4o` 모델 사용하기
  
1. `sentence` 변수에 사투리로 바꾸고 싶은 문장 입력 받기
2. chain1: `sentence`를 충청도 사투리로 변환하는 체인 구성
3. chain2: `sentence`를 경상도 사투리로 변환하는 체인 구성
4. RunnableParallel을 사용하여 실행 

In [21]:
# 실습 코드
model = ChatOpenAI(temperature=0, model_name="gpt-4o")
chain1 = (
    {"sentence": RunnablePassthrough()}
    | PromptTemplate.from_template("{sentence} 를 충청도 사투리로 변환")
    | model
)
chain2 = (
    {"sentence": RunnablePassthrough()}
    | PromptTemplate.from_template("{sentence} 를 경상도 사투리로 변환")
    | model
)

# 병렬 체인 정의 (키워드 매핑 방식)
combined_chain = RunnableParallel(
    chungcheong=chain1,
    gyeongsang=chain2
)

# 실행
sentence = "사투리의 특징적인 억양이나 표현을 사용하여 문장을 만들 수 있습니다."
result = combined_chain.invoke({"sentence": sentence})

# 결과 출력
print("충청도 사투리:", result["chungcheong"])
print("경상도 사투리:", result["gyeongsang"])

충청도 사투리: content='충청도 사투리로 변환하면 다음과 같이 표현할 수 있습니다:\n\n"사투리의 특징적인 억양이나 표현을 써서 문장을 만들 수 있슈."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 42, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CJsxIUzVceFm7obtyBzROvDCPZDoB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c2d01a37-0f80-4a02-bd38-e73fc0d7f165-0' usage_metadata={'input_tokens': 42, 'output_tokens': 41, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
경상도 사투리: content='사투리의 특징적인 억양이나 표현을 사용하여 문장을 만들 수 있데이.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion

### 3. `RunnableLambda`: 사용자 정의 함수 매핑하도록 도와주는 도구


In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def concat_output(text):
    return text['capital'].content + ' ' + text['area'].content

In [ ]:
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_output)} 
            | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 이모지를 넣어줘.")
            | ChatOpenAI())

final_chain.invoke("대한민국")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

In [36]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 대한민국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 출력
print(chain.invoke(3))